In [ ]:
!pip install -q amplpy ampltools pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 15.8 MB/s eta 0:00:00


In [ ]:
# Google Colab & AMPL integration

MODULES, LICENSE_UUID = ["coin", 'gurobi', "cplex", "highs", "gokestrel"], "6b31af08-ff1f-429f-ad0b-4bb913b68f75"

from amplpy import tools
from ampltools import cloud_platform_name, ampl_notebook, register_magics

# instantiate AMPL object and register magics
if cloud_platform_name() is None:
    ampl = AMPL() # Use local installation of AMPL
else:
    ampl = tools.ampl_notebook(modules=MODULES, license_uuid=LICENSE_UUID, g=globals())

register_magics(ampl_object=ampl)

Licensed to Bundle #6300.6669 expiring 20231231: INFO 645 Prescriptive Analytics, Prof. Paul Brooks, Virginia Commonwealth University.


In [ ]:
# Now set the data in AMPL
PROJECTS = [1, 2, 3, 4, 5, 6]
NPV = {1: 15, 2: 5, 3: 13, 4: 14, 5: 20, 6: 9}
Expenditure = {1: 90, 2: 34, 3: 81, 4: 70, 5: 114, 6: 50}
Budget = 220

In [ ]:
ampl.eval ('''

reset;

set PROJECTS;

param NPV {i in PROJECTS};

param Expenditure {i in PROJECTS};

param Budget;

# Decision variables
var x {i in PROJECTS} binary;

# Objective
maximize Total_NPV:
    sum {i in PROJECTS} (NPV[i]* x[i]);

# Constraints
subject to Constraint_Budget:
    sum {i in PROJECTS} (Expenditure[i]*x[i]) <= Budget;

subject to Two_of_1356 {i in {1,3,5,6}}:
    x[1] + x[3] + x[5] + x[6] >= 2;

subject to ThreeFive:
    x[3] = x[5];

subject to Four_Req_Three:
    x[4]<= x[3];

subject to Four_Req_One:
    x[4]<= x[1];

''')

In [ ]:
ampl.set['PROJECTS'] = PROJECTS


ampl.param["NPV"] = NPV
ampl.param["Expenditure"] = Expenditure
ampl.param["Budget"] = Budget


In [ ]:
ampl.setOption('solver', 'cbc')
ampl.solve()

cbc 2.10.10: cbc 2.10.10: optimal solution; objective 33
0 simplex iterations
 


In [ ]:
ampl.eval('''expand;''')

maximize Total_NPV:
	15*x[1] + 5*x[2] + 13*x[3] + 14*x[4] + 20*x[5] + 9*x[6];

subject to Constraint_Budget:
	90*x[1] + 34*x[2] + 81*x[3] + 70*x[4] + 114*x[5] + 50*x[6] <= 220;

subject to Two_of_1356[1]:
	x[1] + x[3] + x[5] + x[6] >= 2;

subject to Two_of_1356[3]:
	x[1] + x[3] + x[5] + x[6] >= 2;

subject to Two_of_1356[5]:
	x[1] + x[3] + x[5] + x[6] >= 2;

subject to Two_of_1356[6]:
	x[1] + x[3] + x[5] + x[6] >= 2;

subject to ThreeFive:
	x[3] - x[5] = 0;

subject to Four_Req_Three:
	-x[3] + x[4] <= 0;

subject to Four_Req_One:
	-x[1] + x[4] <= 0;



In [ ]:
# Print results
obj = ampl.get_objective('Total_NPV')
print("\n")
print("Total_NPV is: ", obj.get().value(), "\n")
print("Projects choosen:")
ampl.display('x');



Total_NPV is:  33.0 

Projects choosen:
x [*] :=
1  0
2  0
3  1
4  0
5  1
6  0
;

